Okay so A few things to note here:
- I am going to have to search for all Supreme Court Cases.
    - I may be able to mitigate this by using the Supreme Court Database ID
- I then need to pull each opinion and code it based on the opinion cluster.
- Opinion clusters give a bit of info I should pull

In [5]:
from bs4 import BeautifulSoup
import re
import csv
import os

Okay so a few things with these xml files
- They code the authors differently. I may not need to pull this information straight from the xml files
- titles that are important are "OPINION" and "DISSENT"
    - Not sure what happens when there are multiple dissents, I will sort that out.
- I can easily remove the citations with this using the \<citation\> and \<page\> tags
- Should I try and remove any Appendix? Not sure at this point.

In [11]:
# Put everything together
files = os.listdir("/Users/peter/misc/briefs/scotusBriefs.xml/opinions-xml")

sep = ','

with open('data/opinions_text.csv', 'w') as f:
    w = csv.DictWriter(f, fieldnames = ('lxs_cite', 'scdb_id', "type", "author", "text")) # define column names
    w.writeheader() # write the header

re_test = re.compile('^((?:Chief\sJustice|Justice|JUSTICE)[^.]*(?:concurring|dissenting)[^.]*\.\n\n)', re.MULTILINE)

for i in files:
    print("Parsing " + i)
    file = open('/Users/peter/misc/briefs/scotusBriefs.xml/opinions-xml' + '/' + i, 'r')
    soup = BeautifulSoup(file, 'xml')

    for o_type in ["OPINION", "DISSENT", "CONCUR"]:
        try:
            names = soup.find(title=o_type)
            athr = names["author"].split('; ') # I need to adjust this, so I can make sure it's not splitting when there's only one author
        except KeyError:
            continue
        except TypeError:
            continue

        # Replace citations with the ones that have a dash in them
        for j in range(len(names.find_all('citation'))):
            modified_citation = names.citation['ref']
            modified_citation = modified_citation.split(sep, 1)[0]
            modified_citation = modified_citation.replace(' ', '-')
            modified_citation = ' ' + modified_citation + ' '
            names.citation.replace_with(modified_citation)

        for k in range(len(names.find_all('page'))):
            modified_citation = names.page['citation']
            modified_citation = modified_citation.split(sep, 1)[0]
            modified_citation = modified_citation.replace(' ', '-')
            modified_citation = ' ' + modified_citation + ' '
            names.page.replace_with(modified_citation)
        
        # This captures lines that either start with 'Chief Justice' or 'Justice'
        # And are only one sentence. As far as I can tell this will do the trick for
        # dealing with multiple dissents or concurrences in a section.

        if len(athr) == 1:
            text = [names.text]
        else:
            text = re.split(re_test, names.text)
            text = text[2::2]

        # Special Cases to fix:
        if i == "2012 U.S. LEXIS 4876.xml":
            if o_type == "OPINION":
                athr = ["111-jgroberts"]
            elif o_type == "DISSENT":
                athr = ["109-rbginsburg", "105-ascalia", "108-cthomas"]
        elif i == "2004 U.S. LEXIS 3386.xml":
            if o_type == "DISSENT":
                athr = ["102-whrehnquist", "105-ascalia", "108-cthomas"]

        try:
            with open('opinions_text_01_13_24.csv', 'a') as f:
                w = csv.DictWriter(f, fieldnames = ('lxs_cite', 'scdb_id', "type", "author", "text")) # define column names
                for l in range(0, len(text)):
                   opinion = {'lxs_cite': soup.find('opinion')['lexisCite'],
                              'scdb_id': soup.find('opinion')['scdb_case'],
                              "type": o_type.lower(),
                              "author": athr[l],
                              "text": text[l]}
                   w.writerow(opinion)
        except IndexError:
            raise IndexError("Error in parsing the " + o_type.lower() + "s for " + i)


Parsing 1970 U.S. LEXIS 88.xml
Parsing 1980 U.S. LEXIS 55.xml
Parsing 1997 U.S. LEXIS 7501.xml
Parsing 1961 U.S. LEXIS 1957.xml
Parsing 1946 U.S. LEXIS 1724.xml
Parsing 1972 U.S. LEXIS 124.xml
Parsing 1955 U.S. LEXIS 1381.xml
Parsing 1988 U.S. LEXIS 2864.xml
Parsing 1981 U.S. LEXIS 53.xml
Parsing 2000 U.S. LEXIS 1538.xml
Parsing 1965 U.S. LEXIS 978.xml
Parsing 1972 U.S. LEXIS 14.xml
Parsing 1981 U.S. LEXIS 47.xml
Parsing 1996 U.S. LEXIS 1549.xml
Parsing 1961 U.S. LEXIS 1943.xml
Parsing 1972 U.S. LEXIS 130.xml
Parsing 1955 U.S. LEXIS 1395.xml
Parsing 1980 U.S. LEXIS 41.xml
Parsing 1973 U.S. LEXIS 12.xml
Parsing 2003 U.S. LEXIS 1738.xml
Parsing 1999 U.S. LEXIS 2346.xml
Parsing 1956 U.S. LEXIS 1803.xml
Parsing 1963 U.S. LEXIS 2497.xml
Parsing 1997 U.S. LEXIS 4034.xml
Parsing 1972 U.S. LEXIS 118.xml
Parsing 2004 U.S. LEXIS 2546.xml
Parsing 1990 U.S. LEXIS 3004.xml
Parsing 1980 U.S. LEXIS 69.xml
Parsing 1977 U.S. LEXIS 146.xml
Parsing 2007 U.S. LEXIS 4337.xml
Parsing 1990 U.S. LEXIS 1175.xm